In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
import matplotlib.pyplot as plt
import numpy as np
import csv
from PIL import Image
import time
import cv2 as cv

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print('Device:', device)
print('Current cuda device:', torch.cuda.current_device())
print('Count of using GPUs:', torch.cuda.device_count())
print(torch.cuda.current_device())
torch.cuda.set_device(1)
print(torch.cuda.current_device())

In [ ]:
xs_0=list()
xs_0_=list()
xs_1=list()
ys_0=list()
data_number= 1
batch_size = 1
epochs = 20000
#100장정도.. 램 가능
with open ('data/Data_narrow.txt', 'rt' ) as r_n:
    target_r = r_n.readline()
    for j in range(data_number): 
        print(j)
        xs_0=list()
        for i in range(2): 
            temp_red=list()
            temp_blue=list()
            temp_green=list()
            
            r=open ('data/'+str(i+20*j)+'_building_r.csv', 'r' )
            rdr=csv.reader(r)
            for target in rdr:
                temp_red.append(target)
                
            r=open ('data/'+str(i+20*j)+'_building_g.csv', 'r' )
            rdr=csv.reader(r)
            for target in rdr:
                temp_green.append(target)
                
            r=open ('data/'+str(i+20*j)+'_building_b.csv', 'r' )
            rdr=csv.reader(r)
            for target in rdr:
                temp_blue.append(target)
                
            temp_red=np.array(temp_red, dtype=np.float32)
            temp_greed=np.array(temp_green, dtype=np.float32)
            temp_blue=np.array(temp_blue, dtype=np.float32)

            xs_1=np.stack((temp_red, temp_green, temp_blue), axis=0)
            xs_1=list(xs_1) 
            xs_0.append(xs_1)
            
            target_r = r_n.readline()
            if i<20:
                A_r,B_r,C_r,D_r,E_r,F_r,G_r,H_r,I_r,J_r,K_r=target_r.split(',')  

                if i==0:
                    pass

                else:
                    if i==1:
                        ys_0.append([float(E_r)])

                    else:

                        ys_0=ys_0[:-1]+[ys_0[-1]+[float(E_r)]]

        xs_0_.append(xs_0)

x_train = np.array(xs_0_, dtype=np.float32)
y_train = np.array(ys_0, dtype=np.float32)
x_train = torch.from_numpy(x_train).cuda()
y_train = torch.from_numpy(y_train).cuda()

xs_0=list()
xs_0_=list()
xs_1=list()
"""
for j in range(data_number): 
    xs_0=list()
    for i in range(10): 
        temp_red=list()
        temp_blue=list()
        temp_green=list()

        r=open ('data/'+str(i+10*j)+'_r.csv', 'r' )
        rdr=csv.reader(r)
        for target in rdr:
            temp_red.append(target)

        r=open ('data/'+str(i+10*j)+'_g.csv', 'r' )
        rdr=csv.reader(r)
        for target in rdr:
            temp_green.append(target)

        r=open ('data/'+str(i+10*j)+'_b.csv', 'r' )
        rdr=csv.reader(r)
        for target in rdr:
            temp_blue.append(target)

        temp_red=np.array(temp_red, dtype=np.float32)
        temp_greed=np.array(temp_green, dtype=np.float32)
        temp_blue=np.array(temp_blue, dtype=np.float32)

        xs_1=np.stack((temp_red, temp_green, temp_blue), axis=0)
        xs_1=list(xs_1) 
        xs_0.append(xs_1)
    
    xs_0_.append(xs_0)
        
x_test = np.array(xs_0_, dtype=np.float32)
x_test = torch.from_numpy(x_test).cuda()
"""
print(x_train.shape,y_train.shape)
dataset = TensorDataset(x_train, y_train)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

#dataset_test = TensorDataset(x_test, y_train)
#dataloader_test = DataLoader(dataset_test, batch_size=batch_size, shuffle=True)
#print(x_test.shape)

xs_0=list()
xs_0_=list()
xs_1=list()
ys_0=list()
temp_red=list()
temp_blue=list()
temp_green=list()

print("data_ready")

In [ ]:
class LRCN(nn.Module):
    def __init__(self):
        super(LRCN, self).__init__()
        self.layer_1 = nn.Conv2d(3,2,5,stride=1)
        self.layer_2 = nn.Conv2d(3,2,5,stride=1)
        #self.layer_3 = nn.Conv2d(10,10,5,stride=2)
        #self.layer_4 = nn.Conv2d(10,10,5,stride=2)
        #self.layer_5 = nn.Conv2d(10,5,5,stride=2)
        #self.layer_6 = nn.LSTM(135,50,1, batch_first = True)
        self.layer_7 = nn.Linear(75264, 1)
        
        self.batch_size = batch_size
        self.length = x_train.shape[1]
        self.relu = nn.ReLU()
        
        self.a=0
        self.b=0
        self.c=0
        self.d=0
        self.e=0
        self.count_=0
        self.o=0
        self.temp_data=0
        self.temp_result=0
        
    def forward(self, x):
        #x = x.reshape((-1, 3, 100, 200))
        x_1 = self.layer_1(x[:,0,:,:,:])
        self.a = self.relu(x_1)
        x_2 = self.layer_2(x[:,1,:,:,:])
        self.b = self.relu(x_2)
        #print(self.a.shape)
        
        self.c = torch.cat((self.a.reshape((self.batch_size, -1)),self.b.reshape((self.batch_size, -1))),axis=1)
        #print(x_3.shape,x_2.shape)
        #reshape((self.batch_size, self.length, x.shape[-3]*x.shape[-2]*x.shape[-1]))
        x= self.layer_7(self.c)
        #print(x.shape)
        
        #x = self.relu(h_n[-1,:,:].reshape((self.batch_size,-1)))
        #x = self.layer_7(self.o[:,1:,:]).reshape((self.batch_size,-1))
        #print(x.shape)
        #x.retain_grad()
        self.count_+=1
        #print("종우",x.grad)
        return x

model = LRCN().cuda()
loss = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4)
#print("서",model.parameters().shape)
      
def train(dataloader, model, loss, optimizer):
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        pred = model(X)
        model.temp_data = X[0]
        print(pred,y)
        loss_result = loss(pred, y)
        optimizer.zero_grad()
        model.temp_result=loss_result
        model.a.retain_grad()
        model.b.retain_grad()
        model.c.retain_grad()
        #model.d.retain_grad()
        #model.e.retain_grad()
        #model.o.retain_grad()
        #print(model.e.grad)
        loss_result.backward()
        #print(model.e.requires_grad)
        #print(model.e.grad)
        #print(model.b.grad)
        #print(model.c.grad)
        #print(model.d.grad)
        #print(model.e.grad)
        #print(loss_result.sum())
        
        
        
        #print("종우",loss_result.register_hook())
    
    #print(result_temp.shape, result_temp)
    #print(result_temp)
    #print(param.shape)
    #print(result_temp.shape)
    #print(model.)
    #print(model.e.grad.shape, result_temp.shape)
    if model.count_==100:
        temp_sum=0
        #print(model.a.grad.shape)
        width_ = model.a.grad.shape[-1]
        result_temp = torch.sum(model.a.grad.detach(),-1)
        height_ = result_temp.shape[-1]
        result_temp = torch.sum(result_temp,-1)
        result_temp_ = torch.sum(result_temp,0)
        result_temp = result_temp_/2
        model.count_=0
        temp_sum=0
        print(result_temp.shape)
        for k in range(1):  
            for j in range(result_temp.shape[0]):
                k=0
                temp_grad = model.a.grad.detach().reshape((model.batch_size, 2, model.a.grad.shape[-2],model.a.grad.shape[-1]))
                temp_sum += result_temp[j]*(temp_grad[0,j,:,:])
            result_ = temp_sum.detach().cpu().numpy()
            min_= np.min(result_)
            result_=result_-min_
            max_ = np.max(result_)
            result_ = ((255*result_)/max_).astype(np.uint8)
            
            pil_image=Image.fromarray(result_)
            #pil_name=str(k)+str(j)+".png"
            result_=pil_image.resize((200,100))
            pil_image.save("name.png")
    
        temp_sum=0
        width_ = model.b.grad.shape[-1]
        result_temp = torch.sum(model.b.grad.detach(),-1)
        height_ = result_temp.shape[-1]
        result_temp = torch.sum(result_temp,-1)
        result_temp_ = torch.sum(result_temp,0)
        result_temp = result_temp_/2
        model.count_=0
        temp_sum=0
        for k in range(1):  
            for j in range(result_temp.shape[0]):
                k=1
                temp_grad = model.b.grad.detach().reshape((model.batch_size, 2, model.b.grad.shape[-2],model.b.grad.shape[-1]))
                temp_sum += result_temp[j]*(temp_grad[0,j,:,:])
            result_ = temp_sum.detach().cpu().numpy()
            min_= np.min(result_)
            result_=result_-min_
            max_ = np.max(result_)
            result_ = ((255*result_)/max_).astype(np.uint8)
            
            pil_image=Image.fromarray(result_)
            #pil_name=str(k)+str(j)+".png"
            result_=pil_image.resize((200,100))
            pil_image.save("name_middle.png")
        
        
        
        #time.sleep(0.001)
        
    optimizer.step()
            
def test(dataloader, model, loss):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss(pred, y).item()
    test_loss /= num_batches
    print(f"Test Error: \n Avg loss: {test_loss:>8f} \n")

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(dataloader, model, loss, optimizer)
    #test(dataloader_test, model, loss)
print("Done!")
